In [2]:
%pip install requests


[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import requests # type: ignore
import json

In [4]:
url_user = "https://dummyjson.com/user"
url_products = "https://dummyjson.com/products"
url_carts = "https://dummyjson.com/carts"

In [5]:
os.makedirs('../data', exist_ok=True)
BASEDIR = os.path.join(os.path.dirname(os.getcwd()), 'data')

In [6]:
DATA_DIR = os.path.join(BASEDIR, 'jsons')
os.makedirs(DATA_DIR, exist_ok=True)

In [7]:
def fetch_and_save_data(url, filename):
    try:
        folder = DATA_DIR
        if not os.path.exists(folder):
            os.makedirs(folder)

        filepath = os.path.join(folder, filename)

        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        data = response.json()

        with open(filepath, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"Data saved to {filepath}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")

In [8]:
fetch_and_save_data(url_user, "users.json")
fetch_and_save_data(url_products, "products.json")
fetch_and_save_data(url_carts, "carts.json")

Data saved to /home/reuben/Documents/WORK/SAVANNAH/data/jsons/users.json
Data saved to /home/reuben/Documents/WORK/SAVANNAH/data/jsons/products.json
Data saved to /home/reuben/Documents/WORK/SAVANNAH/data/jsons/carts.json
